<a href="https://colab.research.google.com/github/iasminqmoura/ia_2023/blob/main/Trabalho_05_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Carrega os dados dos carros
nomes_colunas = ['preco_compra', 'preco_manutencao', 'num_portas', 'num_pessoas', 'tamanho_bagageiro', 'seguranca', 'classe']
dados_carros = pd.read_csv('car_data.csv', names=nomes_colunas)

# Pré-processamento dos dados: Codificação one-hot para as colunas categóricas
codificador_one_hot = OneHotEncoder(sparse_output=False)
caracteristicas_categoricas = dados_carros.drop('classe', axis=1)
caracteristicas_categoricas_codificadas = codificador_one_hot.fit_transform(caracteristicas_categoricas)
nomes_caracteristicas_codificadas = codificador_one_hot.get_feature_names_out(caracteristicas_categoricas.columns)
dados_carros_codificados = pd.DataFrame(caracteristicas_categoricas_codificadas, columns=nomes_caracteristicas_codificadas)

# Adiciona a coluna classe novamente ao conjunto de dados
dados_carros_codificados = pd.concat([dados_carros_codificados, dados_carros[['classe']]], axis=1)

# Divide os dados em características (X) e rótulo (y)
X = dados_carros_codificados.drop('classe', axis=1)
y = dados_carros_codificados['classe']

# Normaliza as características numéricas
normalizador = StandardScaler()
X_normalizado = normalizador.fit_transform(X)

# Codificação one-hot para o rótulo
y_codificado = codificador_one_hot.fit_transform(y.values.reshape(-1, 1))

# Divide os dados em conjuntos de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X_normalizado, y_codificado, test_size=0.2, random_state=42)

# Define a arquitetura da rede neural para a classificação do conjunto de dados de carros
modelo = Sequential([
    Dense(64, activation='relu', input_shape=(X_treino.shape[1],)),
    Dense(32, activation='relu'),
    Dense(y_codificado.shape[1], activation='softmax')
])

# Compila o modelo
modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treina o modelo e armazena o histórico de treinamento
historico = modelo.fit(X_treino, y_treino, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

# Avalia o modelo no conjunto de teste e exibe os resultados
perda, precisao = modelo.evaluate(X_teste, y_teste, verbose=0)

# Cria um DataFrame para os resultados
resultados = pd.DataFrame({
    'Conjunto': ['Treino', 'Teste'],
    'Perda': [historico.history['loss'][-1], perda],
    'Precisao (%)': [historico.history['accuracy'][-1] * 100, precisao * 100]
})

# Exibe uma amostra do conjunto de dados com formatação
print("Exemplo do Dataset de Carros:")
display(dados_carros_codificados.head().style.set_properties(**{'text-align': 'left'}).set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}]))

# Apresentação da tabela de resultados
print("Resultados do Treinamento:")
resultados.style.set_properties(**{'text-align': 'left'}).set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])


Exemplo do Dataset de Carros:


,preco_compra_high,preco_compra_low,preco_compra_med,preco_compra_vhigh,preco_manutencao_high,preco_manutencao_low,preco_manutencao_med,preco_manutencao_vhigh,num_portas_2,num_portas_3,num_portas_4,num_portas_5more,num_pessoas_2,num_pessoas_4,num_pessoas_more,tamanho_bagageiro_big,tamanho_bagageiro_med,tamanho_bagageiro_small,seguranca_high,seguranca_low,seguranca_med,classe
0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,unacc
1,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,unacc
2,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,unacc
3,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,unacc
4,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,unacc


Resultados do Treinamento:


,Conjunto,Perda,Precisao (%)
0,Treino,0.001552,100.000000
1,Teste,0.063570,98.554915
